In [129]:
!pip install pybaseball

In [130]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix
from pybaseball import statcast
from pybaseball import playerid_lookup, statcast_pitcher
import matplotlib.pyplot as plt
import seaborn as sns

In [131]:
start_date = '2023-04-01'
end_date = '2023-10-01'
data = statcast(start_dt=start_date, end_dt=end_date)
selected_columns = [
    'pitch_type', 'player_name', 'stand', 'p_throws', 
    'balls', 'strikes', 'game_year', 'outs_when_up', 'inning', 
    'pitch_number', 'bat_score', 
    'fld_score', 'if_fielding_alignment', 'of_fielding_alignment'
]
filtered_data = data[selected_columns]
pitcher_data = filtered_data[filtered_data['player_name'] == 'Manaea, Sean']
pitcher_data

This is a large query, it may take a moment to complete


/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|                                                                                                                        | 0/184 [00:00<?, ?it/s]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecate

SSLError: HTTPSConnectionPool(host='baseballsavant.mlb.com', port=443): Max retries exceeded with url: /statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7CPO%7CS%7C=&hfSea=&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=2023-05-18&game_date_lt=2023-05-18&team=&position=&hfRO=&home_road=&hfFlag=&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=h_launch_speed&sort_order=desc&min_abs=0&type=details& (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

In [ ]:
pitcher_data = pitcher_data.copy()

In [ ]:
pitcher_data.shape

In [ ]:
pitcher_data['pitch_type'].value_counts()

In [ ]:
pitcher_data['if_fielding_alignment'].value_counts()

In [ ]:
pitcher_data['of_fielding_alignment'].value_counts()

In [ ]:

pitch_counts = pitcher_data.groupby(['inning', 'pitch_type']).size().unstack(fill_value=0)

pitch_percentage = pitch_counts.div(pitch_counts.sum(axis=1), axis=0) * 100

ax = pitch_percentage.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='viridis')

plt.title('Percentage of Pitch Types per Inning')
plt.xlabel('Inning')
plt.ylabel('Percentage')
plt.legend(title='Pitch Type', bbox_to_anchor=(1.05, 1), loc='upper left')

for container in ax.containers:
    ax.bar_label(
        container, 
        labels=[f'{val:.1f}%' if val > 0 else '' for val in container.datavalues], 
        label_type='center'
    )

plt.tight_layout()
plt.show()


In [ ]:

pitch_counts_by_inning = pitcher_data.groupby(['inning', 'pitch_type']).size().unstack(fill_value=0)

colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A8', '#FF9633', '#33FFF3', '#A833FF', '#FFFF33', '#33FF9E']

fig, axes = plt.subplots(3, 3, figsize=(18, 18))

for i, ax in enumerate(axes.flatten(), start=1):
    if i in pitch_counts_by_inning.index: 
        pitch_counts = pitch_counts_by_inning.loc[i]
        wedges, texts, autotexts = ax.pie(
            pitch_counts, 
            labels=pitch_counts.index, 
            autopct='%1.1f%%', 
            startangle=90, 
            colors=colors[:len(pitch_counts)]
        )
        for text in texts:
            text.set_fontsize(10)
        for autotext in autotexts:
            autotext.set_fontsize(10)
        ax.set_title(f'Inning {i}', fontsize=14, fontweight='bold')
    else:
        ax.axis('off')  

fig.suptitle('Pitch Type Percentage by Inning (Sean Manaea)', fontsize=20, fontweight='bold')

fig.legend(
    pitch_counts_by_inning.columns, 
    loc='lower center', 
    ncol=5, 
    bbox_to_anchor=(0.5, -0.05), 
    fontsize=12
)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])

plt.show()

summary_table = pitch_counts_by_inning.T

import ace_tools as tools; tools.display_dataframe_to_user(name="Pitch Type Counts by Inning")



In [ ]:
le_stand = LabelEncoder()
le_p_throws = LabelEncoder()
le_if_align = LabelEncoder()
le_of_align = LabelEncoder()
pitcher_data['stand'] = le_stand.fit_transform(pitcher_data['stand'])
pitcher_data['p_throws'] = le_p_throws.fit_transform(pitcher_data['p_throws'])
pitcher_data['if_fielding_alignment'] = le_if_align.fit_transform(pitcher_data['if_fielding_alignment'])
pitcher_data['of_fielding_alignment'] = le_of_align.fit_transform(pitcher_data['of_fielding_alignment'])

In [ ]:
X = pitcher_data.drop(['pitch_type', 'player_name'], axis=1)
y = pitcher_data['pitch_type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
def predict_pitch_type(outs_when_up, inning, balls, strikes, Pitch_number, Bat_score, Fld_score):
    current_game = pd.DataFrame([{
        'stand': le_stand.transform(['L'])[0],
        'p_throws': le_p_throws.transform(['L'])[0],
        'balls': balls,
        'strikes': strikes,
        'game_year': 2023,
        'outs_when_up': outs_when_up,
        'inning': inning,
        'pitch_number': Pitch_number,
        'bat_score': Bat_score,
        'fld_score': Fld_score,
        'if_fielding_alignment': le_if_align.transform(['Standard'])[0],
        'of_fielding_alignment': le_of_align.transform(['Standard'])[0]
    }])
    
    probabilities = model.predict_proba(current_game)[0]

    pitch_type_probs = dict(zip(model.classes_, probabilities))
    
    return pitch_type_probs

outs_when_up = 2
inning = 8
balls = 2
strikes = 2
Pitch_number = 4
Bat_score = 9
Fld_score = 7

In [ ]:
pitch_type_probs = predict_pitch_type(outs_when_up, inning, balls, strikes, Pitch_number, Bat_score, Fld_score)

In [ ]:
pitch_type_probs_df = pd.DataFrame(
        pitch_type_probs.items(), columns=['Pitch Type', 'Probability'])
pitch_type_probs_df

In [ ]:
pitch_type_probs_df = pd.DataFrame(
        pitch_type_probs.items(), columns=['Pitch Type', 'Probability']
    ).sort_values(by='Probability', ascending=False).reset_index(drop=True)
pitch_type_probs_df

In [ ]:
start_date = '2023-04-01'
end_date = '2023-10-01'
data_pitch_speed = statcast(start_dt=start_date, end_dt=end_date)
selected_columns = [
    'pitch_type', 'player_name', 'stand', 'p_throws', 
    'balls', 'strikes', 'game_year', 'outs_when_up', 'inning', 
    'pitch_number', 'bat_score', 'effective_speed',
    'fld_score', 'if_fielding_alignment', 'of_fielding_alignment'
]
filtered_data = data_pitch_speed[selected_columns]
effective_speed_data = filtered_data[filtered_data['player_name'] == 'Manaea, Sean']
effective_speed_data

In [ ]:

plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")

sns.histplot(
    data=effective_speed_data, 
    x='effective_speed', 
    hue='pitch_type', 
    kde=True,
    palette='bright', 
    multiple='stack'
)

plt.title('Distribution of Effective Speed by Pitch Type (Sean Manaea)', fontsize=16)
plt.xlabel('Effective Speed (mph)', fontsize=12)
plt.ylabel('Count', fontsize=12)

plt.show()


In [ ]:
def safe_transform(encoder, value):
    """Safely transforms a value using the encoder.
    If the value is not found in the encoder's classes_, return -1."""
    if value in encoder.classes_:
        return encoder.transform([value])[0]
    else:
        return -1

effective_speed_data['stand'] = effective_speed_data['stand'].astype(str)
effective_speed_data['p_throws'] = effective_speed_data['p_throws'].astype(str)
effective_speed_data['if_fielding_alignment'] = effective_speed_data['if_fielding_alignment'].astype(str)
effective_speed_data['of_fielding_alignment'] = effective_speed_data['of_fielding_alignment'].astype(str)
effective_speed_data['pitch_type'] = effective_speed_data['pitch_type'].astype(str)

le_stand.fit(effective_speed_data['stand'])
le_p_throws.fit(effective_speed_data['p_throws'])
le_if_align.fit(effective_speed_data['if_fielding_alignment'])
le_of_align.fit(effective_speed_data['of_fielding_alignment'])
le_pitch_type.fit(effective_speed_data['pitch_type'])

effective_speed_data['stand'] = le_stand.transform(effective_speed_data['stand'])
effective_speed_data['p_throws'] = le_p_throws.transform(effective_speed_data['p_throws'])
effective_speed_data['if_fielding_alignment'] = le_if_align.transform(effective_speed_data['if_fielding_alignment'])
effective_speed_data['of_fielding_alignment'] = le_of_align.transform(effective_speed_data['of_fielding_alignment'])
effective_speed_data['pitch_type'] = le_pitch_type.transform(effective_speed_data['pitch_type'])

X = effective_speed_data.drop(['effective_speed', 'player_name'], axis=1)
y = effective_speed_data['effective_speed']

X = X.apply(pd.to_numeric, errors='coerce')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

speed_model = RandomForestRegressor(n_estimators=100, random_state=42)
speed_model.fit(X_train, y_train)

def predict_effective_speed_by_pitch_type(outs_when_up, inning, balls, strikes, Pitch_number, Bat_score, Fld_score):
    pitch_types = effective_speed_data['pitch_type'].unique() 
    pitch_type_predictions = {} 

    for pitch_type in pitch_types:
        current_game = pd.DataFrame([{
            'pitch_type': pitch_type,
            'stand': safe_transform(le_stand, 'L'),
            'p_throws': safe_transform(le_p_throws, 'L'),
            'balls': balls,
            'strikes': strikes,
            'game_year': 2023,
            'outs_when_up': outs_when_up,
            'inning': inning,
            'pitch_number': Pitch_number,
            'bat_score': Bat_score,
            'fld_score': Fld_score,
            'if_fielding_alignment': safe_transform(le_if_align, 'Standard'),
            'of_fielding_alignment': safe_transform(le_of_align, 'Standard')
        }])

        predicted_speed = speed_model.predict(current_game)[0]
        pitch_type_predictions[le_pitch_type.inverse_transform([pitch_type])[0]] = predicted_speed

    predictions_df = pd.DataFrame(
        pitch_type_predictions.items(), columns=['Pitch Type', 'Predicted Effective Speed']
    ).sort_values(by='Predicted Effective Speed', ascending=False).reset_index(drop=True)

    return predictions_df

outs_when_up = outs_when_up
inning = inning
balls = balls
strikes = strikes
Pitch_number = Pitch_number
Bat_score = Bat_score
Fld_score = Fld_score
predictions_df = predict_effective_speed_by_pitch_type(
    outs_when_up, inning, balls, strikes, Pitch_number, Bat_score, Fld_score
)

predictions_df


In [ ]:
pitch_type_probs_df

In [ ]:
merged_df = pd.merge(pitch_type_probs_df, predictions_df, on='Pitch Type')

In [ ]:
merged_df